# Advance Autogen Concepts

## Selector Group Chat

In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [40]:
api_key = "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"
model_client_2 = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3-0324:free",
    api_key=api_key,
    model_info={
             "family": "deepseek",
             "vision": True,
             "function_calling": True,
             "json_output": False,
         }
)


/Users/mayank/Live-Class/Agentic-1.0/Autogen/autogen-lc/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [ ]:
planning_agent = AssistantAgent(
    name='PlanningAgent',
    description = 'An agent for planning tasks, this agent should be first to engage when given a new task',
    model_client=model_client,
    system_message='''
    You are a planning agent
    your job is to break down complex tasks into smaller, manageable substasks.
    You team members are :
    WebSearchAgent : Searches for Information.
    DataAnalystAgent : Perform Claculation.

    Your only plan and delegate tasks - you don not execute yourself.

    When assigning the tasks, use the below format :
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and print out 'TERMINATE'.
    '''
)

In [55]:
search_web_tool('how many runs I hit in 2006-2007 in cricket')

' \n        Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade : 1397 points\n        James Posey : 550 points        \n    '

In [46]:
def search_web_tool(query:str)->str:
    ''' 
    '''
    # Simulating Web search
    if ('2006-2007') in query:
        return ''' 
        Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade : 1397 points
        James Posey : 550 points        
    '''
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [47]:
def percentage_change_tool(start:float,end:float) ->float:
    if(start==0):
        return 0
    return ((end-start)/start)*100

data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.
    ''',
)

In [29]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination
combined_termination = TextMentionTermination('TERMINATE') | MaxMessageTermination(max_messages=15)

{roles} {history} {description}


Available fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.



In [30]:
selector_prompt = '''  
Select and agent to perform the task.
{roles}

Current conversation history:
{history}

Read the above conversation and then select and agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
only select one agent.

'''

In [ ]:
from autogen_agentchat.teams import SelectorGroupChat

selector_team = SelectorGroupChat(
    participants=[planning_agent,web_search_agent,data_analyst_agent],
    model_client=model_client_2,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)


In [13]:
task = ' Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his total' \
'rebounds between 2007-2008 and 2008-2009 seasons ?'

In [14]:
from autogen_agentchat.ui import Console

stream = selector_team.run_stream(task=task)

In [15]:
await Console(stream)

---------- TextMessage (user) ----------
 Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?
---------- TextMessage (PlanningAgent) ----------
To address the inquiry, we will take the following steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Find the total rebounds for that player in the 2007-2008 NBA season.
3. Find the total rebounds for that player in the 2008-2009 NBA season.
4. Calculate the percentage change in total rebounds between 2007-2008 and 2008-2009.

Tasks split for the agents:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Look up the total rebounds for that player in the 2007-2008 NBA season.
3. WebSearchAgent: Look up the total rebounds for that player in the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage ch

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content=' Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=163, completion_tokens=215), metadata={}, content='To address the inquiry, we will take the following steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Find the total rebounds for that player in the 2007-2008 NBA season.\n3. Find the total rebounds for that player in the 2008-2009 NBA season.\n4. Calculate the percentage change in total rebounds between 2007-2008 and 2008-2009.\n\nTasks split for the agents:\n\n1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent: Look up the total rebounds for that player in the 200

In [24]:
from autogen_agentchat.ui import Console

stream = selector_team.run_stream(task=task)
await Console(stream)

---------- TextMessage (user) ----------
 Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?
---------- TextMessage (PlanningAgent) ----------
To answer the question, we need to break down the task as follows:

1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Determine the total number of rebounds that player had in the 2007-2008 NBA season.
3. Determine the total number of rebounds that player had in the 2008-2009 NBA season.
4. Calculate the percentage change in total rebounds from the 2007-2008 season to the 2008-2009 season.

Tasks for agents:

1. WebSearchAgent: Verify and provide details of the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Look up and confirm the total rebounds for that player in the 2007-2008 NBA season.
3. WebSearchAgent: Look up and confirm the total rebounds

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content=' Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=692, completion_tokens=231), metadata={}, content='To answer the question, we need to break down the task as follows:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Determine the total number of rebounds that player had in the 2007-2008 NBA season.\n3. Determine the total number of rebounds that player had in the 2008-2009 NBA season.\n4. Calculate the percentage change in total rebounds from the 2007-2008 season to the 2008-2009 season.\n\nTasks for agents:\n\n1. WebSearchAgent: Verify and provide details of the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2.

In [51]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence,List

def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        print('-'*50)
        print(f"The agent which we got last was {messages[-1].source}")
        return planning_agent.name
    return None

from autogen_agentchat.teams import SelectorGroupChat

selector_team_2 = SelectorGroupChat(
    participants=[planning_agent,web_search_agent,data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func
)


In [53]:
selector_team_2.reset()

<coroutine object BaseGroupChat.reset at 0x10fe51030>

In [54]:
from autogen_agentchat.ui import Console

stream_2 = selector_team_2.run_stream(task=task)
await Console(stream_2)

--------------------------------------------------
The agent which we got last was user
---------- TextMessage (user) ----------
 Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?
---------- TextMessage (PlanningAgent) ----------
To answer this, let's proceed step-by-step:

1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Find out the total rebounds for that player in the 2007-2008 and 2008-2009 NBA seasons.
3. Calculate the percentage change in rebounds between these two seasons.

From previous information:

- The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, scoring 1,397 points.
- In the 2007-2008 season, Dwyane Wade had 214 total rebounds.
- In the 2008-2009 season, he had 398 total rebounds.

To find the percentage change in rebounds:
- Percentage change = ((398 - 214) / 214) * 100 
- 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content=' Who was the Miami heat player with the highest point in 2006-2007 season and what was the percentage change in his totalrebounds between 2007-2008 and 2008-2009 seasons ?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=1507, completion_tokens=239), metadata={}, content="To answer this, let's proceed step-by-step:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Find out the total rebounds for that player in the 2007-2008 and 2008-2009 NBA seasons.\n3. Calculate the percentage change in rebounds between these two seasons.\n\nFrom previous information:\n\n- The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, scoring 1,397 points.\n- In the 2007-2008 season, Dwyane Wade had 214 total rebounds.\n- In the 2008-2009 season, he had 398 total rebounds.\n\nTo find the percentag